In [1]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

from dreamUtils import *
from dreamNetworks import *


Using TensorFlow backend.


DreamNetworks is ready!


## DATA LOAD

In [2]:
DATA_DIR = './data-test/'
channels_coord = DATA_DIR + 'channelcoords.mat'
files = os.listdir(DATA_DIR)
files.remove('channelcoords.mat')

In [3]:
# files_labels = {'file': ['JL02trial_1.mat','JL02trial_2.mat','JL02trial_3.mat','JL02trial_4.mat',
#                        'JL02trial_5.mat','JL02trial_6.mat','JL02trial_7.mat','JL02trial_8.mat',
#                        'JL02trial_9.mat','JL02trial_10.mat','JL02trial_11.mat','JL02trial_12.mat', 
#                        'JL02trial_13.mat','JL02trial_14.mat','JL02trial_15.mat','JL02trial_16.mat',
#                        'JL02trial_17.mat','JL02trial_18.mat','JL02trial_19.mat','JL02trial_20.mat',
#                        'JL02trial_21.mat','JL02trial_22.mat','JL02trial_23.mat', 'JL02trial_24.mat'],
#               'label': [2,2,2,2,2,2,2,1,2,2,0,2,2,2,1,2,1,2,2,2,2,2,2,2]}

files_labels = {'file': ['JL02trial_8.mat','JL02trial_11.mat'],
              'label': [1,0]}

labels_df =  pd.DataFrame.from_dict(files_labels).set_index('file')

In [4]:
locs_3D = sio.loadmat(channels_coord)['locstemp']
print(locs_3D.shape)
locs_2D = map_to_2d(locs_3D)
print(locs_2D.shape)

(256, 3)
(256, 2)


In [5]:
all_data = []
all_data_labels = []

for single_trial in labels_df.index:
    label = labels_df.get_value(single_trial, 'label')
    #take last 20 seconds and 256 electrodes
    #referecence electrode is excluded
    a_trial = ((sio.loadmat(DATA_DIR + single_trial)['a_trial']).T)[-10000:,0:256]
    all_data.append(a_trial)
    all_data_labels.append(label)

In [6]:
a_trial = sio.loadmat('/home/ezgi/Desktop/SemesterProject_EPFL/EEGLearn/Sample data/FeatureMat_timeWin.mat')

In [7]:
a_trial['features'].shape

(2670, 1345)

In [8]:
len(all_data)

2

In [12]:
import h5py

In [15]:
with h5py.File('./data/H009_E1_NREM_S01.mat','r') as f:
    print(f.keys())

KeysView(<HDF5 file "H009_E1_NREM_S01.mat" (mode r)>)


In [18]:
arrays = {}
f = h5py.File('./data/H009_E1_NREM_S01.mat')
for k, v in f.items():
    arrays[k] = np.array(v)

In [21]:
arrays['datavr'].shape

(394125, 257)

In [10]:
a_trial = sio.loadmat(DATA_DIR + 'JL02trial_3.mat')['a_trial']

In [11]:
a_trial

array([[  5.90494   ,   5.456661  ,   3.7931693 , ..., -46.28712   ,
        -48.23105   , -46.852055  ],
       [  5.761282  ,   5.3931656 ,   4.0317554 , ..., -43.170185  ,
        -44.438335  , -42.915382  ],
       [  4.797691  ,   4.529368  ,   3.3924654 , ..., -35.585415  ,
        -36.068577  , -34.327198  ],
       ...,
       [  0.70536757,   1.0350038 ,  -1.6873348 , ..., -38.038395  ,
        -40.55295   , -38.59091   ],
       [  1.4886457 ,   1.6795789 ,  -0.9567653 , ..., -38.62174   ,
        -41.44178   , -39.558346  ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]], dtype=float32)

### Image Data Creation

In [8]:
# high bias !!!
labels_df.label.value_counts()

1    1
0    1
Name: label, dtype: int64

In [9]:
def lab(y, sample_size):
    return [y]*sample_size

In [10]:
# assign each image the label of trial
# here, 10000 image is taken from each trial is known
labels= np.concatenate( [lab(y, 10000) for y in all_data_labels]  )

In [11]:
# create an image per time step
data = np.concatenate([gen_images(locs_2D, X , 32, normalize=False,
                augment=False, pca=False, std_mult=0.1, n_components=2, edgeless=False) for X in all_data])

Interpolated 10000/10000

In [137]:
X_train, X_test, y_train, y_test = train_test_split(data, labels,test_size=0.2, random_state=1453)

In [138]:
# Make labels categorical
y_train_32 = np_utils.to_categorical(y_train)
y_test_32 = np_utils.to_categorical(y_test)

In [139]:
X_train.shape

(16000, 1, 32, 32)

In [141]:
y_train_32

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [103]:
model = CNN_Image32()  # TODO::Forget to compile
# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [104]:
np.unique(y_test[:,2], return_counts=True)

(array([0., 1.], dtype=float32), array([2012,  988]))

In [121]:
#correct input shape before feeding to CNN
X_train_32 = X_train.reshape(-1,32,32,1)
X_test_32 = X_test.reshape(-1,32,32,1)

In [106]:
# train the model
model.fit(X_train_32, y_train, validation_data=(X_test_32, y_test), epochs=3)

Train on 12000 samples, validate on 3000 samples
Epoch 1/3
12000/12000 [==============================] - 100s 8ms/step - loss: 10.2670 - acc: 0.3599 - val_loss: 10.4714 - val_acc: 0.3503
Epoch 2/3
12000/12000 [==============================] - 104s 9ms/step - loss: 10.3142 - acc: 0.3601 - val_loss: 10.4714 - val_acc: 0.3503
Epoch 3/3
12000/12000 [==============================] - 102s 9ms/step - loss: 10.3142 - acc: 0.3601 - val_loss: 10.4714 - val_acc: 0.3503


In [49]:
all_data_labels

[1, 2, 2, 0, 2, 2, 2, 1, 2, 1]

In [144]:
 # create model
model = Sequential()
# add layers
# activation is NONE  
#model.add(Conv2D(32, kernel_size=3, activation='relu'))

model.add(Conv2D(32, kernel_size=3, input_shape=(32,32,1), 
                 kernel_initializer='glorot_uniform', activation='relu'))
model.add(Conv2D(64, kernel_size=3, padding='same', kernel_initializer='glorot_uniform', activation='relu'))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    

In [145]:
# train the model
model.fit(X_train_32, y_train_32, validation_data=(X_test_32, y_test_32), epochs=3)

Train on 16000 samples, validate on 4000 samples
Epoch 1/3
16000/16000 [==============================] - 55s 3ms/step - loss: 8.0859 - acc: 0.4976 - val_loss: 7.8858 - val_acc: 0.5108
Epoch 2/3
16000/16000 [==============================] - 57s 4ms/step - loss: 8.1024 - acc: 0.4973 - val_loss: 7.8858 - val_acc: 0.5108
Epoch 3/3
16000/16000 [==============================] - 59s 4ms/step - loss: 8.1024 - acc: 0.4973 - val_loss: 7.8858 - val_acc: 0.5108
